## Intro

### Scoring Logic Description

1. We start by importing the odis dataframe from a CSV that includes all the relevant datapoint to score and display data
2. We compute the scores for each criteria specific to the commune (independant from subject)
3. We compute the scores for each criteria specific to the subject (dependand from both subject and commune) 
4. We identify all commune<->neighbour pairs (binômes) for each commune within search radius
5. We compute category scores (emploi, logement, education etc...) as an average of the all the scores for a given category
6. For each commune we compare the commune and neighbour category scores and weighted the highest one with category weights defined by subject and then keep the best weighted score for each commune
8. We display result in on a map

In [1]:
# THIS SHOULD BE THE END OF JUPYTER NOTEBOOK EXPORT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy import stats
import folium as flm #required for gdf.explore()
import shapely as shp
from shapely.wkt import loads
from shapely.geometry import Polygon
from sklearn import preprocessing

### 1. Fetching key indicators from ODIS source file

In [2]:
def init_loading_datasets(odis_file, scores_cat_file, metiers_file, formations_file, ecoles_file, maternites_file, sante_file):
    odis = gpd.GeoDataFrame(gpd.read_parquet(odis_file))
    odis.set_geometry(odis.polygon, inplace=True)
    odis = odis[~odis.polygon.isna()]

    # Index of all scores and their explanations
    scores_cat = pd.read_csv(scores_cat_file)

    #Later we need the code FAP <-> FAP Name used to classify jobs
    codfap_index = pd.read_csv(metiers_file, delimiter=';')

    # Later we need the code formation <-> Formation Name used to classify trainings
    # source: https://www.data.gouv.fr/fr/datasets/liste-publique-des-organismes-de-formation-l-6351-7-1-du-code-du-travail/
    codformations_index = pd.read_csv(formations_file).set_index('codformation')

    # Etablissements scolaires
    annuaire_ecoles = pd.read_parquet(ecoles_file)
    annuaire_ecoles.geometry = annuaire_ecoles.geometry.apply(shp.from_wkb)

    #Annuaire Maternités
    # Source: https://www.data.gouv.fr/fr/datasets/liste-des-maternites-de-france-depuis-2000/
    annuaire_maternites = pd.read_csv(maternites_file, delimiter=';')
    annuaire_maternites.drop_duplicates(subset=['FI_ET'], keep='last', inplace=True)
    annuaire_maternites.head()

    # Annuaire etablissements santé
    # Source: https://www.data.gouv.fr/fr/datasets/reexposition-des-donnees-finess/
    annuaire_sante = pd.read_parquet(sante_file)
    annuaire_sante = annuaire_sante[annuaire_sante.LibelleSph == 'Etablissement public de santé']
    annuaire_sante['geometry'] = gpd.points_from_xy(annuaire_sante.coordxet, annuaire_sante.coordyet, crs='epsg:2154')
    annuaire_sante = pd.merge(annuaire_sante, annuaire_maternites[['FI_ET']], left_on='nofinesset', right_on='FI_ET', how='left', indicator="maternite")
    annuaire_sante.drop(columns=['FI_ET'], inplace=True)
    annuaire_sante.maternite = np.where(annuaire_sante.maternite == 'both', True, False)
    annuaire_sante['codgeo'] = annuaire_sante.Departement + annuaire_sante.Commune

    return odis, scores_cat, codfap_index, codformations_index, annuaire_ecoles, annuaire_sante

### 2. Distance filter + Gathering nearby Communes Scores

In [3]:
# Filtering dataframe based on subject distance preference (to save on compute time later on)
def filter_loc_by_distance(df, distance):
    return df[df.dist_current_loc < distance * 1000]

# Put None as a score in the monome case
def monome_cleanup(df):
    mask = ~df['binome']
    for col in df.columns:
        if col.endswith('_binome'):
            df.loc[mask, col] = None
    return df

In [4]:
def adding_score_voisins(df_search, scores_cat):
    #df_search is the dataframe pre-filtered by location
    #df_source is the dataframe with all the communes
    binome_columns = ['codgeo','libgeo','polygon','epci_code','epci_nom'] + scores_cat[scores_cat.incl_binome]['score'].to_list()+scores_cat[scores_cat.incl_binome]['metric'].to_list()
    binome_columns = list(set(binome_columns) & set(df_search.columns))
    df_binomes = df_search[binome_columns].copy()

    # Adds itself to list of voisins = monome case
    # Note: this code triggers the SettingWithCopyWarning but I don't know how to fix it...
    df_search.codgeo_voisins = df_search.apply(lambda x: np.append(x.codgeo_voisins, x.codgeo), axis=1)

    # Explodes the dataframe to have a row for each voisins + itself
    df_search['codgeo_voisins_copy'] = df_search['codgeo_voisins']
    df_search_exploded = df_search.explode('codgeo_voisins_copy')
    
    # For each commune (codgeo) in search area (df_search) we add all its voisin's scores
    odis_search_exploded = pd.merge(df_search_exploded, df_binomes.add_suffix('_binome'), left_on='codgeo_voisins_copy', right_on='codgeo_binome', how='left')
    
    # Adds a column to identify binomes vs monomes + cleanup
    odis_search_exploded['binome'] = odis_search_exploded.apply(lambda x: False if x.codgeo == x.codgeo_binome else True, axis=1)
    odis_search_exploded.drop(columns={'codgeo_voisins_copy'}, inplace=True)

    #We remove all values for the monome case to avoid accounting for them in the category score calculation
    odis_search_exploded = monome_cleanup(odis_search_exploded)

    return odis_search_exploded

### 3. Criterias Scoring

In [5]:
#Computing distance from current commune 
#Using a crs that allows to compute distance in meters for metropolitan France

def distance_calc(df, ref_point):
    return int(df.distance(ref_point))

def add_distance_to_current_loc(df, current_codgeo):
    projected_crs = "EPSG:2154"
    zone_recherche = gpd.GeoDataFrame(df[df.codgeo == current_codgeo]['polygon'])
    zone_recherche.set_geometry('polygon', inplace=True)
    zone_recherche.to_crs(projected_crs, inplace=True)

    df.to_crs(projected_crs, inplace=True)
    df['dist_current_loc'] = df['polygon'].apply(distance_calc, ref_point=zone_recherche.iloc[0].polygon)
    return df

In [6]:
#Adding score specific to subject looking for a job identified as en besoin
def codes_match(df, codes_list):
    #returns a list of codfaps that matches
    if df is None:
        return []
    return list(set(df.tolist()).intersection(set(codes_list)))

def fap_names_lookup(df):
    return list(codfap_index[codfap_index['Code FAP 341'].isin(df)]['Intitulé FAP 341'])

In [7]:
def compute_criteria_scores(df, prefs): 
    df = df.copy()
    
    # Using QuantileTransfer to normalize all scores between 0 and 1 for the region
    t = preprocessing.QuantileTransformer(output_distribution="uniform")

    #met_ration est le ratio d'offres non-pourvues pour 1000 habitants
    df['met_ratio']= 1000 * df.met/df.pop_be
    df['met_scaled'] = t.fit_transform(df[['met_ratio']].fillna(0))
    #met_tension_ratio est le ratio d'offres population de la zone (pour 1000 habitants)
    df['met_tension_ratio'] = 1000 * df.met_tension/df.pop_be
    df['met_tension_scaled'] = t.fit_transform(df[['met_tension_ratio']].fillna(0))
    #svc_ratio est le ratio de services d'inclusion de la commune (pour 1000 habitants)
    df['svc_incl_ratio'] = 1000 * df.svc_incl_count/df.pop_be
    df['svc_incl_scaled'] = t.fit_transform(df[['svc_incl_ratio']].fillna(0))
    #log_vac_ratio est le ratio de logements vacants de la commune % total logements
    df['log_vac_ratio'] = df.log_vac/df.log_total
    df['log_vac_scaled'] = t.fit_transform(df[['log_vac_ratio']].fillna(0))
    #pol est le score selon la couleur politique (extreme droite = 0, gauche = 1)
    df['pol_scaled'] = df[['pol_num']].astype('float')
    
    if prefs['hebergement'] == "Chez l'habitant":
        #log_5p+_ratio est le ratio de residences principales de 5 pièces ou plus % total residences principales
        df['log_5p_ratio'] = df['rp_5+pieces']/df.log_rp
        df['log_5p_scaled'] = t.fit_transform(df[['log_5p_ratio']].fillna(0))

    if len(prefs['classe_enfants']) > 0: 
        # Risque de fermeture école: ratio de classe à risque de fermeture % nombre d'écoles
        df['risque_fermeture_ratio'] = df.risque_fermeture/df.ecoles_ct
        df['classes_ferm_scaled'] = t.fit_transform(df[['risque_fermeture_ratio']].fillna(0))


    # Subject Specific criterias

    # We compute the distance from the current location 
    df['reloc_dist_scaled'] = (1-df['dist_current_loc']/(prefs['loc_distance_km']*1000))
    df['reloc_epci_scaled'] = np.where(df['epci_code'] == df[df.codgeo == prefs['commune_actuelle']]['epci_code'].iloc[0],1,0)

    #For each adult we look for jobs categories that match what is needed
    i=1
    for adult in range(0,prefs['nb_adultes']):
        if len(prefs['codes_metiers'][adult]) > 0:
            df['met_match_codes_adult'+str(i)] = df.be_codfap_top.apply(codes_match, codes_list=prefs['codes_metiers'][adult])
            df['met_match_adult'+str(i)] = df['met_match_codes_adult'+str(i)].apply(len)
            df['met_match_adult'+str(i)+'_scaled'] = t.fit_transform(df[['met_match_adult'+str(i)]].fillna(0))
            i+=1

    j=1
    for adult in range(0,prefs['nb_adultes']):
        if len(prefs['codes_formations'][adult]) > 0:
            df['form_match_codes_adult'+str(j)] = df.codes_formations.apply(codes_match, codes_list=prefs['codes_formations'][adult])
            df['form_match_adult'+str(j)] = df['form_match_codes_adult'+str(j)].apply(len)
            df['form_match_adult'+str(j)+'_scaled'] = t.fit_transform(df[['form_match_adult'+str(j)]].fillna(0))
        j+=1

    
    return df

### 4. Category Scoring

In [8]:
def compute_cat_scores(df, scores_cat, penalty):
    df = df.copy()
    df_binome = pd.DataFrame()
    columns_in_use = set(df.columns) & set(scores_cat.score)
    columns_in_use_binome = set(df.columns) & set([score+'_binome' for score in scores_cat.score])
    for cat in set(scores_cat.cat):
        cat_scores_indices = [score for score in scores_cat[scores_cat['cat'] == cat]['score'] if score in columns_in_use]
        cat_scores_indices_binome = [score+'_binome' for score in scores_cat[scores_cat['cat'] == cat]['score'] if score+'_binome' in columns_in_use_binome]

        # Efficiently select all relevant rows at once
        cat_scores_df = df[cat_scores_indices]
        for col in cat_scores_indices_binome:
            mask = df[col].notna()
            df_binome[col] = pd.to_numeric(df[col], errors='coerce')
            df_binome.loc[mask, col] = df.loc[mask, col] * (1-penalty) 
            cat_scores_df = pd.concat([cat_scores_df, df_binome[col]], axis=1)
        df[cat + '_cat_score'] = cat_scores_df.astype(float).mean(axis=1)

    return df

### 5. Final Binome Score Weighted

In [9]:
def compute_binome_score_old(df, binome_penalty, prefs):
    scores_col = [col for col in df.columns if col.endswith('_cat_score')]
    max_scores = pd.DataFrame()
    
    for col in scores_col:
        cat_weight = prefs[col.split('_')[0]]
        max_scores[col] = cat_weight * np.where(
            df[col] >= (1-binome_penalty)*df[col+'_binome'],
            df[col],
            (1-binome_penalty)*df[col+'_binome']
            )
    
    return max_scores.mean(axis=1).round(1)


In [10]:
def compute_binome_score(df, scores_cat, prefs):
    scores_cat_col = [col for col in df.columns if col.endswith('_cat_score')]
    weighted_scores = pd.DataFrame()
    for col in scores_cat_col:
        cat_weight =  prefs['poids_'+col.split('_')[0]]
        weighted_scores[col] = cat_weight * df[col]
    
    return weighted_scores.astype(float).mean(axis=1)

In [11]:
def best_score_compute(df):
    #Keeping the best (top #1) monome or binome result for each commune
    best = df.sort_values('weighted_score', ascending=False).groupby('codgeo').head(1)
    return best

In [12]:
#Main function that aggregates most of the above in one sequence
def compute_odis_score(df, scores_cat, prefs):
    df = add_distance_to_current_loc(df, current_codgeo=prefs['commune_actuelle'])

    # We filter by distance to reduce the compute cost on a smaller odis_search dataframe
    odis_search = filter_loc_by_distance(df, distance=prefs['loc_distance_km'])

    # We compute the subject specific scores
    odis_scored = compute_criteria_scores(odis_search, prefs=prefs)

    # We add the criteria scores for all neighbor communes forming monomes and binomes
    odis_exploded = adding_score_voisins(odis_scored, scores_cat)

    # We compute the category scores for both the target and the binome
    odis_exploded = compute_cat_scores(odis_exploded, scores_cat=scores_cat, penalty=prefs['binome_penalty'])

    # We computing the final weighted score for all commune<->voisin combinations
    odis_exploded['weighted_score'] = compute_binome_score(odis_exploded, scores_cat=scores_cat, prefs=prefs)

    # We keep best monome or binome for each commune 
    odis_search_best = best_score_compute(odis_exploded)

    return odis_search_best


### 6. Generating Narrative

Here we want to generate a 'human readable' explanation about why scored high a given location.
Things to show:
- Target commune name and EPCI
- Weighted Score
- If Binome, show the binome and EPCI if different from target
- Show top 3 criterias target (weighted ?) 
- Show top 3 criterias for binome (weighted ?)

In [13]:
# THIS SHOULD BE THE END OF JUPYTER NOTEBOOK EXPORT

## Export to Python file for streamlit

In [14]:
%save -f -r ../streamlit/odisscoring.py 1-13
# This saves the cells 0 to 22 (and their execution history unfortunately) to a python file that I can use in Streamlit
# Make sure to restart before running this cell
# Don't forget to restart streamlit after this

The following commands were written to file `../streamlit/odisscoring.py`:
# THIS SHOULD BE THE END OF JUPYTER NOTEBOOK EXPORT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy import stats
import folium as flm #required for gdf.explore()
import shapely as shp
from shapely.wkt import loads
from shapely.geometry import Polygon
from sklearn import preprocessing
def init_loading_datasets(odis_file, scores_cat_file, metiers_file, formations_file, ecoles_file, maternites_file, sante_file):
    odis = gpd.GeoDataFrame(gpd.read_parquet(odis_file))
    odis.set_geometry(odis.polygon, inplace=True)
    odis = odis[~odis.polygon.isna()]

    # Index of all scores and their explanations
    scores_cat = pd.read_csv(scores_cat_file)

    #Later we need the code FAP <-> FAP Name used to classify jobs
    codfap_index = pd.read_csv(metiers_file, delimiter=';')

    # Later we need the code formation <-> Formation Name used to classify trainings

In [60]:
# Restart and run all the cells above this one

## Notebook explorations

In [20]:
# Init
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

ODIS_FILE = '../csv/odis_april_2025_jacques.parquet'
SCORES_CAT_FILE = '../csv/odis_scores_cat.csv'
METIERS_FILE = '../csv/dares_nomenclature_fap2021.csv'
FORMATIONS_FILE = '../csv/index_formations.csv'
ECOLES_FILE = '../csv/annuaire_ecoles_france_mini.parquet'
MATERNITE_FILE = '../csv/annuaire_maternites_DREES.csv'
SANTE_FILE = '../csv/annuaire_sante_finess.parquet'

odis, scores_cat, codfap_index, codformations_index, annuaire_ecoles, annuaire_sante = init_loading_datasets(
    odis_file=ODIS_FILE,
    scores_cat_file=SCORES_CAT_FILE,
    metiers_file=METIERS_FILE,
    formations_file=FORMATIONS_FILE,
    ecoles_file=ECOLES_FILE,
    maternites_file=MATERNITE_FILE,
    sante_file=SANTE_FILE
    )


In [ ]:
# Subject preferences weighted score computation
prefs = {
    'poids_emploi':2,
    'poids_logement':1,
    'poids_education':1,
    'poids_soutien':1,
    'poids_mobilité':0,
    'commune_actuelle':'33281',
    'hebergement':"Chez l'habitant",
    'logement':'Location',
    'loc_distance_km':25,
    'codes_metiers':{
        'codes_metiers_adulte1':['S1X40','J0X33','A1X41'],
        'codes_metiers_adulte2':['T4X60','T2A60']
    },
    'codes_formations':{
        'codes_formations_adulte1':[423],
        'codes_formations_adulte2':[315,100]
    },
    'classe_enfants':['Maternelle', 'Collège'],
    'besoin_sante': None,
    'binome_penalty':0.1
}

In [47]:
# Step by Step Execution
from time import time

def performance_tracker(t, text, timer_mode):
    if timer_mode:
        print(str(round(time()-t,2))+'|'+text)
        return time()
t = time()
timer_mode = True

df = odis
score_cat = scores_cat
prefs = prefs
#
df = add_distance_to_current_loc(df, current_codgeo=prefs['commune_actuelle'])
t = performance_tracker(t, 'Add Distance End', timer_mode)

# We filter by distance to reduce the compute cost on a smaller odis_search dataframe
odis_search = filter_loc_by_distance(df, distance=prefs['loc_distance_km'])
t = performance_tracker(t, 'Filter Loc by Distance', timer_mode)

# We compute the subject specific scores
odis_scored = compute_criteria_scores(odis_search, prefs=prefs)
t = performance_tracker(t, 'Compute Subject Score End', timer_mode)

# We add the criteria scores for all neighbor communes forming monomes and binomes
odis_exploded = adding_score_voisins(odis_scored, scores_cat)
t = performance_tracker(t, 'Adding Score Voisin', timer_mode)

# We compute the category scores for both the target and the binome
odis_exploded = compute_cat_scores(odis_exploded, scores_cat=scores_cat, penalty=prefs['binome_penalty'])
t = performance_tracker(t, 'Compute Cat Score End', timer_mode)

# We computing the final weighted score for all commune<->voisin combinations
odis_exploded['weighted_score'] = compute_binome_score(odis_exploded, scores_cat=scores_cat, prefs=prefs)
t = performance_tracker(t, 'Compute Weighted Score End', timer_mode)

# We keep best monome or binome for each commune 
odis_search_best = best_score_compute(odis_exploded)
t = performance_tracker(t, 'Compute Best Score End', timer_mode)


0.7|Add Distance End
0.01|Filter Loc by Distance
0.14|Compute Subject Score End


/home/jbesancon/d4g/13_odis/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (157). n_quantiles is set to n_samples.
  warnings.warn(
/home/jbesancon/d4g/13_odis/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (157). n_quantiles is set to n_samples.
  warnings.warn(
/home/jbesancon/d4g/13_odis/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (157). n_quantiles is set to n_samples.
  warnings.warn(
/home/jbesancon/d4g/13_odis/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (157). n_quantiles is set to n_samples.
  warnings.warn(
/home/jbesancon/d4g/13_odis/.venv/lib/python3.11/site-packages/sklea

0.12|Adding Score Voisin
0.13|Compute Cat Score End
0.01|Compute Weighted Score End
0.02|Compute Best Score End


In [21]:
annuaire_sante.head()

,nofinesset,nofinessej,RaisonSociale,RaisonSocialeLongue,ComplementRaisonSociale,ComplementDistribution,NumeroVoie,TypeVoie,LibelleVoie,ComplementVoie,LieuDitBp,Commune,Departement,LibelleDepartement,LigneAcheminement,Telephone,Telecopie,Categorie,LibelleCategorie,CategorieAgregat,LibelleCategorieAgregat,Siret,CodeApe,CodeMft,LibelleMft,CodeSph,LibelleSph,DateOuverture,DateAutorisation,DateMajStructure,NumeroUAI,Emetteur,coordxet,coordyet,sourcecoordet,datemaj,geometry,maternite,codgeo
0,010000024,010780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,None,None,900,RTE,DE PARIS,None,None,451,01,AIN,01440 VIRIAT,0474454647,0474454114,355,Centre Hospitalier (C.H.),1102,Centres Hospitaliers,26010004500012,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1979-02-13,1979-02-13,2020-02-04,None,None,870199.8750,6571579.0,"1,ATLASANTE,81,BAN_14-05-2025,L93_METROPOLE",2025-05-07,POINT (870199.875 6571579),True,01451
1,010000032,010780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,None,None,700,AV,DE NARVIK,None,BP 139,034,01,AIN,01300 BELLEY,0479425959,0479425996,355,Centre Hospitalier (C.H.),1102,Centres Hospitaliers,26010003700068,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2021-07-07,None,None,908313.3125,6520061.0,"1,ATLASANTE,81,BAN_14-05-2025,L93_METROPOLE",2025-05-07,POINT (908313.312 6520061),True,01034
2,010000065,010780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,None,None,14,R,DE L'HOPITAL,None,BP 615,427,01,AIN,01606 TREVOUX CEDEX,0474105000,0474105019,355,Centre Hospitalier (C.H.),1102,Centres Hospitaliers,26010028400017,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2018-01-12,None,None,837272.3125,6539470.5,"2,ATLASANTE,100,BDADRESSE_v2.2_03-2024,L93_MET...",2025-05-07,POINT (837272.312 6539470.5),False,01427
3,010000081,010780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,None,None,160,R,MARC PANISSOD,None,BP 437,173,01,AIN,01174 GEX CEDEX,0450403838,0450403825,106,"Centre hospitalier, ex Hôpital local",1106,Hôpitaux Locaux,26010010200011,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2020-02-04,None,None,935201.8750,6584824.5,"1,ATLASANTE,100,BDADRESSE_v2.2_03-2024,L93_MET...",2025-05-07,POINT (935201.875 6584824.5),False,01173
4,010000099,010780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,None,None,13,AV,DU DOCTEUR BOYER,None,None,244,01,AIN,01800 MEXIMIEUX,0474610177,0474347184,106,"Centre hospitalier, ex Hôpital local",1106,Hôpitaux Locaux,26010013600019,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1945-01-01,1945-01-01,2020-06-30,None,None,870118.1875,6536469.5,"1,ATLASANTE,85,BAN_14-05-2025,L93_METROPOLE",2025-05-07,POINT (870118.188 6536469.5),False,01244


In [22]:
annuaire_sante.value_counts(['LibelleCategorieAgregat', 'Categorie'])

LibelleCategorieAgregat                                      Categorie
Autres Etablissements de Lutte contre les Maladies Mentales  156          1527
Centres Hospitaliers                                         355          1339
Centres Hospitaliers Spécialisés Lutte Maladies Mentales     292          1183
Etablissements de Soins de Longue Durée                      362           511
Autres Etablissements de Lutte contre les Maladies Mentales  425           415
Centres Hospitaliers Régionaux                               101           374
Hôpitaux Locaux                                              106           198
Autres Etablissements Relevant de la Loi Hospitalière        696           113
Etablissements de Formation des Personnels Sanitaires        300            83
Autres Etablissements de Lutte contre les Maladies Mentales  412            78
                                                             366            33
                                                            

In [50]:
hebergement_options = ["Chez l'habitant", 'Location', 'Foyer']
hebergement_index = hebergement_options.index("Chez l'habitant")
hebergement_index

0

In [57]:
codfap_index.head()

,Code FAP 22,Intitulé FAP 22,Code FAP 86,Intitulé FAP 86,Code FAP 228,Intitulé FAP 228,Code FAP 341,Intitulé FAP 341
0,A,"Agriculture, marine, pêche",A0X,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0X41,Eleveurs,A0X41c,Eleveurs de lapins et volailles
1,A,"Agriculture, marine, pêche",A0X,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0X41,Eleveurs,A0X41d,Eleveurs d'ovins ou de caprins
2,A,"Agriculture, marine, pêche",A0X,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0X41,Eleveurs,A0X41e,Eleveurs de porcins
3,A,"Agriculture, marine, pêche",A0X,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0X42,"Bûcherons, sylviculteurs et agents forestiers",A0X42b,"Sylviculteurs, agents forestiers et agents de ..."
4,A,"Agriculture, marine, pêche",A1X,"Maraîchers, jardiniers, viticulteurs",A1X40,Maraîchers et horticulteurs,A1X40,Maraîchers et horticulteurs


In [ ]:
codfap_index[codfap_index['Intitulé FAP 341'].str.contains('menuis')]['Intitulé FAP 341']

Code FAP 22                                                         B
Intitulé FAP 22                             Bâtiment, travaux publics
Code FAP 86                                                       B2X
Intitulé FAP 86                  Ouvriers du second œuvre du bâtiment
Code FAP 228                                                    B2X38
Intitulé FAP 228    Ouvriers qualifiés en menuiserie et en agencem...
Code FAP 341                                                    B2X38
Intitulé FAP 341    Ouvriers qualifiés en menuiserie et en agencem...
Name: 256, dtype: object

,nofinesset,nofinessej,RaisonSociale,RaisonSocialeLongue,ComplementRaisonSociale,ComplementDistribution,NumeroVoie,TypeVoie,LibelleVoie,ComplementVoie,...,DateOuverture,DateAutorisation,DateMajStructure,NumeroUAI,Emetteur,coordxet,coordyet,sourcecoordet,datemaj,geometry
0,010000024,010780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,None,None,900,RTE,DE PARIS,None,...,1979-02-13,1979-02-13,2020-02-04,None,None,870199.8750,6571579.0,"1,ATLASANTE,81,BAN_14-05-2025,L93_METROPOLE",2025-05-07,POINT (870199.875 6571579)
1,010000032,010780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,None,None,700,AV,DE NARVIK,None,...,1901-01-01,1901-01-01,2021-07-07,None,None,908313.3125,6520061.0,"1,ATLASANTE,81,BAN_14-05-2025,L93_METROPOLE",2025-05-07,POINT (908313.312 6520061)
2,010000065,010780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,None,None,14,R,DE L'HOPITAL,None,...,1901-01-01,1901-01-01,2018-01-12,None,None,837272.3125,6539470.5,"2,ATLASANTE,100,BDADRESSE_v2.2_03-2024,L93_MET...",2025-05-07,POINT (837272.312 6539470.5)
3,010000081,010780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,None,None,160,R,MARC PANISSOD,None,...,1901-01-01,1901-01-01,2020-02-04,None,None,935201.8750,6584824.5,"1,ATLASANTE,100,BDADRESSE_v2.2_03-2024,L93_MET...",2025-05-07,POINT (935201.875 6584824.5)
4,010000099,010780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,None,None,13,AV,DU DOCTEUR BOYER,None,...,1945-01-01,1945-01-01,2020-06-30,None,None,870118.1875,6536469.5,"1,ATLASANTE,85,BAN_14-05-2025,L93_METROPOLE",2025-05-07,POINT (870118.188 6536469.5)


,ANNEE,NOM_MAT,TYPE,STATUT,FI_ET,FI_EJ,COM,NOMCOM,CPO,ADRESSE,LIT_OBS,SALTRAV,ACCTOT
328,2014,CLINIQUE DE MONTBELIARD,1,PRIVE LUCRATIF,250011723,250017126,25388,MONTBELIARD,25205,11 AV LEON BLUM,19,3,352
378,2013,Clinique PASTEUR-LANROZE,1,PRIVE LUCRATIF,290000140,290000447,29019,BREST,29229,32 R AUGUSTE KERVERN,23,4,851
1134,2016,NOUVELLE CLINIQUE DES TROIS FRONTIERES,1,PRIVE LUCRATIF,680020088,680020062,68297,SAINT-LOUIS,68300,8 R SAINT DAMIEN,10,2,326
2085,2014,CENTRE HOSPITALIER DE FALAISE,1,PUBLIC,140000233,140000118,14258,FALAISE,14700,BD DES BERCAGNES,19,2,394
2091,2013,CH COTE FLEURIE - SITE DE CRICQUEBOEUF,1,PUBLIC,140026410,140026279,14202,CRICQUEBOEUF,14113,RTE DEPARTEMENTALE 62,20,4,486


In [15]:
annuaire_sante.value_counts('maternite')

maternite
False    101054
True        530
Name: count, dtype: int64

In [12]:
annuaire_sante2[annuaire_sante2.maternite][['RaisonSociale','LibelleCategorieAgregat','geometry']].explore(column='LibelleCategorieAgregat', categorical=True)

In [42]:
scores_cat_prefs=scores_cat.copy()
for index, row in scores_cat.iterrows():
    if row.loc['incl_binome']:
        row_to_add = scores_cat.loc[[index]]
        row_to_add['score'] = row_to_add['score'] + '_binome'
        row_to_add['incl_binome'] = False
        scores_cat_prefs = pd.concat([scores_cat_prefs, row_to_add])
scores_cat_prefs['weight'] = scores_cat_prefs['cat'].apply(lambda x: prefs[x])
scores_cat_prefs
# row_to_add

,score,score_name,metric,unit,tooltip,cat,incl_binome,high_value_adj,weight
0,met_scaled,Taux Besoin Emploi,met_ratio,besoin/1000 hab,Emplois non pourvus pour 1000 habitants (Sourc...,emploi,True,élevé,2
1,met_tension_scaled,Taux Besoin Emploi en Tension,met_tension_ratio,besoin/1000 hab,Emplois en tension non pourvus pour 1000 habit...,emploi,True,élevé,2
2,svc_incl_scaled,Taux Services Inclusion,svc_incl_ratio,services/1000 hab,Services d'inclusions pour 1000 habitants,soutien,True,élevé,1
3,log_vac_scaled,Taux Logements Vacants,log_vac_ratio,/Total Logements,NaN,logement,True,élevé,1
4,log_5p_scaled,Taux Grandes Résidences Principales,log_5p_ratio,/Total Résidences Principales,NaN,logement,True,élevé,1
5,classes_ferm_scaled,Taux Classes à Risque de Fermeture,risque_fermeture_ratio,/Total Ecoles,Ratio du nombre de classes à riques de fermetu...,education,True,élevé,1
6,pol_scaled,Couleur Politique de la commmue,pol_num,NaN,Affiliation à un parti politique en faveur de ...,soutien,False,favorable,1
7,met_match_adult1_scaled,Match compétences et Besoin Emploi Adult 1,met_match_adult1,familles d'emplois,Nombre de familles d'emplois dans le top 10,emploi,True,présent,2
8,met_match_adult2_scaled,Match compétences et Besoin Emploi Adult 2,met_match_adult2,familles d'emplois,Nombre de familles d'emplois dans le top 10,emploi,True,présent,2
9,form_match_adult1_scaled,Match compétences et Centres de formation,form_match_adult1,sites de formations,Nombre centres de formations dans la commune,emploi,False,présent,2


In [222]:
odis_exploded['weighted_score'] = compute_binome_score(odis_exploded, prefs=prefs)
odis_exploded['weighted_score']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
257   NaN
258   NaN
259   NaN
260   NaN
261   NaN
Name: weighted_score, Length: 262, dtype: float64

In [217]:
odis_exploded

,codgeo,libgeo,typecom,reg_code,reg_nom,dep_code,dep_nom,epci_code,epci_nom,niveau_equipements_services,...,log_5p_ratio_binome,risque_fermeture_ratio_binome,met_match_adult1_binome,met_match_adult2_binome,binome,emploi_cat_score,logement_cat_score,mobilité_cat_score,education_cat_score,soutien_cat_score
0,33013,Artigues-près-Bordeaux,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,2.0,...,0.541127,3.000000,1,2,True,59.857774,22.045486,56.795,NaN,51.443415
1,33013,Artigues-près-Bordeaux,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,2.0,...,0.488789,3.000000,1,2,True,59.857774,13.600667,56.795,NaN,52.711918
2,33013,Artigues-près-Bordeaux,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,2.0,...,0.237933,2.571429,1,2,True,59.396063,17.342812,56.795,NaN,55.589616
3,33013,Artigues-près-Bordeaux,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,2.0,...,0.162542,2.888889,1,2,True,59.501168,15.070191,56.795,NaN,57.916730
4,33013,Artigues-près-Bordeaux,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,2.0,...,0.154327,3.000000,1,2,True,59.857774,10.901214,56.795,NaN,58.096502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,33550,Villenave-d'Ornon,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,3.0,...,0.377520,3.000000,1,2,True,58.845000,10.099740,82.405,NaN,53.894568
258,33550,Villenave-d'Ornon,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,3.0,...,0.512268,2.400000,1,2,True,58.278183,15.418924,82.405,NaN,54.882438
259,33550,Villenave-d'Ornon,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,3.0,...,0.355415,2.636364,1,2,True,58.400351,8.780973,82.405,NaN,59.439321
260,33550,Villenave-d'Ornon,COM,75,Nouvelle-Aquitaine,33,Gironde,243300316,Bordeaux Métropole,3.0,...,0.187113,2.764706,1,2,True,58.456597,10.379816,82.405,NaN,59.184391


In [ ]:

penalty=0.1
df = odis_exploded.copy()
df_binome = pd.DataFrame()
columns_in_use = set(df.columns) & set(scores_cat.score)
columns_in_use_binome = set(df.columns) & set([score+'_binome' for score in scores_cat.score])
for cat in set(scores_cat.cat):
    cat_scores_indices = [score for score in scores_cat[scores_cat['cat'] == cat]['score'] if score in columns_in_use]
    cat_scores_indices_binome = [score+'_binome' for score in scores_cat[scores_cat['cat'] == cat]['score'] if score+'_binome' in columns_in_use_binome]

    # Efficiently select all relevant rows at once
    cat_scores_df = df[cat_scores_indices]
    for col in cat_scores_indices_binome:
        mask = df[col].notna()
        df_binome[col] = pd.to_numeric(df[col], errors='coerce')
        df_binome.loc[mask, col] = df.loc[mask, col] * (1-penalty) 
        cat_scores_df = pd.concat([cat_scores_df, df_binome[col]], axis=1)
    df[cat + '_cat_score'] = 100 * cat_scores_df.astype(float).mean(axis=1)


In [214]:
df.head(1).to_csv('../csv/test.csv')

In [ ]:




# We provide the scores columns as a parameter to compute faster
#scores_col = [col for col in odis_exploded.columns if col.endswith('_cat_score')]

# We computing the final weighted score for all commune<->voisin combinations
# odis_exploded['weighted_score'] = compute_binome_score(
#     odis_exploded,
#     binome_penalty=prefs['binome_penalty'],
#     weights=prefs
#     )

# We keep best monome or binome for each commune 
odis_search_best = best_score_compute(odis_exploded)

In [25]:
# Showing results on an interactive map
cols_to_show = (
        ['codgeo','libgeo','weighted_score','binome','libgeo_binome','dist_current_loc','polygon']
        +[col for col in odis_search_best.columns if '_codes_' in col]
        +[col for col in odis_search_best.columns if '_cat_score' in col]
        )
#odis_search_best[cols_to_show].explore('weighted_score', popup=True)
#odis_search_best.plot('weighted_score')
#odis_search_best[cols_to_show].explore('weighted_score', tooltip=True)

In [62]:
scores_cat

,score,score_name,metric,unit,tooltip,cat,incl_binome,high_value_adj
0,met_scaled,Taux Besoin Emploi,met_ratio,besoin/1000 hab,Emplois non pourvus pour 1000 habitants (Sourc...,emploi,True,élevé
1,met_tension_scaled,Taux Besoin Emploi en Tension,met_tension_ratio,besoin/1000 hab,Emplois en tension non pourvus pour 1000 habit...,emploi,True,élevé
2,svc_incl_scaled,Taux Services Inclusion,svc_incl_ratio,services/1000 hab,Services d'inclusions pour 1000 habitants,soutien,True,élevé
3,log_vac_scaled,Taux Logements Vacants,log_vac_ratio,/Total Logements,NaN,logement,True,élevé
4,log_5p_scaled,Taux Grandes Résidences Principales,log_5p_ratio,/Total Résidences Principales,NaN,logement,True,élevé
5,classes_ferm_scaled,Taux Classes à Risque de Fermeture,risque_fermeture_ratio,/Total Ecoles,Ratio du nombre de classes à riques de fermetu...,emploi,True,élevé
6,pol_scaled,Couleur Politique de la commmue,pol_num,NaN,Affiliation à un parti politique en faveur de ...,soutien,False,favorable
7,met_match_adult1_scaled,Match compétences et Besoin Emploi Adult 1,met_match_adult1,familles d'emplois,Nombre de familles d'emplois dans le top 10,emploi,True,présent
8,met_match_adult2_scaled,Match compétences et Besoin Emploi Adult 2,met_match_adult2,familles d'emplois,Nombre de familles d'emplois dans le top 10,emploi,True,présent
9,form_match_adult1_scaled,Match compétences et Centres de formation,form_match_adult1,sites de formations,Nombre centres de formations dans la commune,emploi,False,présent


## 9. Export to SuperSet

In [27]:
# def concatenate_strings(row):
#   return '{"type": "Feature","geometry":' + shp.to_geojson(row['polygon']) + '}'


# odis_search_best_export = gpd.GeoDataFrame(odis_search_best.copy())
# odis_search_best_export.set_geometry(odis_search_best_export.polygon, crs='EPSG:2154', inplace=True)
# odis_search_best_export.to_crs(epsg=4326, inplace=True)
# odis_search_best_export["polygon_as_json"] = odis_search_best_export.apply(concatenate_strings, axis=1)
# odis_search_best_export.drop(['polygon','polygon_binome'], axis=1, inplace=True)

# cols = ['met_match_codes','met_match_codes_binome','be_codfap_top','be_libfap_top','codgeo_voisins_binome','pitch']
# for col in cols:
#     odis_search_best_export[col] = odis_search_best_export[col].apply(lambda x: x.tolist() if type(x) == np.ndarray else x)

In [28]:
# from sqlalchemy import create_engine, text

# db_host = "localhost"  # Replace with the actual host (e.g., 'superset_db' if in the same Docker network, or 'localhost' if exposed)
# db_port = "5433"  # Replace with the actual port (usually 5432)
# db_user = "superset"  # Replace with the database user (often 'superset')
# db_password = "superset"  # Replace with the database password
# db_name = "examples"  # Replace with the database name (often 'superset')

# engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [29]:
# table_name = "odis_stream2_result"  # Choose a name for the table in PostgreSQL
# odis_search_best_export.to_sql(table_name, engine, if_exists='replace', schema='public', index=False)
# sql = text("GRANT SELECT ON odis_stream2_result TO examples")

# with engine.begin() as connection:
#     connection.execute(sql)

# print(f"DataFrame successfully written to table '{table_name}' in the Superset database.")

Note to myself:
Après avoir importé les données dans Postgres il faut donner les droits au user 'examples' sur la table
> docker exec -it superset_db psql -h superset_db -p 5432 -U superset -d examples

> GRANT SELECT ON odis_stream2_result TO examples;

> GRANT USAGE ON SCHEMA public TO examples;
